In [ ]:
!cp '/content/drive/MyDrive/corrected data.csv' '/content'
!cp -r '/content/drive/MyDrive/property classf data' '/content'

In [ ]:
import pandas as pd
from glob import glob
import shutil
import random
import numpy as np
from tqdm.notebook import tqdm
import os

In [ ]:
df = pd.read_csv('/content/corrected data.csv')

In [ ]:
df

In [ ]:
print(len(set(list(df['oid']))), len(list(df['oid'])))

2254 2314


In [ ]:
(list(df['Duplicate'])).count(2)

120

In [ ]:
def mdir(path):
  os.makedirs(path, exist_ok = True)
def rmdir(path):
  shutil.rmtree(path)

# Move all images from train, val, test to single folder with classes

In [ ]:
# rmdir('images')
mdir('all_images')
mdir('images')
mdir('images/com')
mdir('images/hoa')
mdir('images/res')
# rmdir('images')

In [ ]:
test_pth = glob('/content/property classf data/test/*/*')
train_pth = glob('/content/property classf data/train/*/*')
val_pth = glob('/content/property classf data/val/*/*')

print(len(test_pth) + len(train_pth) + len(val_pth))

2314


In [ ]:
all_pth = test_pth + train_pth + val_pth
for i in tqdm(all_pth):
  src = i
  dst = '/content/all_images/'+i.split('/')[-1]
  shutil.copy(src, dst)

  0%|          | 0/2314 [00:00<?, ?it/s]

In [ ]:
print(len(glob('/content/all_images/*')))

2254


In [ ]:
dups = []
cls_map = {'Comm':'com', 'HOA':'hoa', 'Resi':'res'}
for i in df.iterrows():
  oid = i[1]['oid']
  dup = i[1]['Duplicate']
  cls = i[1]['correct_class']
  if(oid not in dups):
    dups.append(oid)
    src = '/content/all_images/'+oid+'.png'
    dst = '/content/images/'+cls_map[cls]+'/'+oid+'.png'
    shutil.copy(src, dst)
  else:
    pass

In [ ]:
print(len(glob('/content/images/*/*')))

2254


# Correcting class labels for Feb month

In [ ]:
!cp -r '/content/drive/MyDrive/property classf data/additional data for feb month/png_clipped' '/content'

In [ ]:
!cp '/content/drive/MyDrive/property classf data/additional data for feb month/Classification activity_2 - Sheet1.csv' '/content'

In [ ]:
feb_df = pd.read_csv('/content/Classification activity_2 - Sheet1.csv')

In [ ]:
feb_df

,S.No,oid,actual class/es,correct class,Remarks
0,1,63d440516407576a1d098f61,['HoA'],HOA,NaN
1,2,63d484706407576a1d098f73,['Industrial unit'],COMM,NaN
2,3,63d4ede46ba37bea43f6f359,['HoA'],HOA,NaN
3,4,63d51a756407576a1d0990f1,['HoA'],HOA,NaN
4,5,63d5217e6407576a1d0990ff,['HoA'],HOA,NaN
...,...,...,...,...,...
910,911,64001e8e74e20d4f7b5cd0ef,['Residential'],RES,NaN
911,912,64002310b6e513fcbfe80a53,['HoA'],HOA,NaN
912,913,6400322842a6ea491be1a7dc,['HoA'],HOA,NaN
913,914,64003269b6e513fcbfe80a55,['Office'],COMM,NaN


In [ ]:
dups = []
cls_map = {'COMM':'com', 'HOA':'hoa', 'RES':'res'}
for i in feb_df.iterrows():
  oid = i[1]['oid']
  cls = i[1]['correct class']
  if(oid not in dups):
    dups.append(oid)
    src = '/content/png_clipped/'+oid+'.png'
    dst = '/content/images/'+cls_map[cls]+'/'+oid+'.png'
    shutil.copy(src, dst)
  else:
    pass

In [ ]:
print(len(glob('/content/images/*/*')))

3168


In [ ]:
for i in glob('/content/images/*'):
  print(i.split('/')[-1], len(glob(i+'/*')))

res 470
hoa 899
com 1799


In [ ]:
pth = glob('/content/images/*/*')

In [ ]:
random.shuffle(pth)

# Splitting data into train, val and test with 70:20:10 split

In [ ]:
total_len = len(pth)
train_len = int(total_len*0.7)
rem_len = total_len - train_len
val_len = int(rem_len*2.0/3.0)
test_len = int(rem_len/3)

print(100*train_len/total_len, 100*val_len/total_len, 100*test_len/total_len)

69.98106060606061 20.012626262626263 10.006313131313131


In [ ]:
mdir('train')
mdir('train/com')
mdir('train/hoa')
mdir('train/res')

mdir('test')
mdir('test/com')
mdir('test/hoa')
mdir('test/res')

mdir('val')
mdir('val/com')
mdir('val/hoa')
mdir('val/res')

In [ ]:
train_pth = pth[0:train_len]
val_pth = pth[train_len:train_len+val_len]
test_pth = pth[train_len+val_len:]

In [ ]:
len(train_pth)+len(val_pth)+len(test_pth)

3168

# Move to train, test and val folders

In [ ]:
def movettv(pth, type):
  for i in tqdm(pth):
    cls = i.split('/')[-2]
    img = i.split('/')[-1]
    src = i
    dst = '/content/'+type+'/'+cls+'/'+img
    shutil.copy(src, dst)

In [ ]:
movettv(train_pth, 'train')
movettv(val_pth, 'val')
movettv(test_pth, 'test')

  0%|          | 0/2217 [00:00<?, ?it/s]

  0%|          | 0/634 [00:00<?, ?it/s]

  0%|          | 0/317 [00:00<?, ?it/s]

# Applying augmentations

In [ ]:
total_data = glob('/content/train/*/*')

In [ ]:
!rm -r train_aug
!mkdir train_aug
!mkdir train_aug/res
!mkdir train_aug/com
!mkdir train_aug/hoa

In [ ]:
for i in total_data:
  src = i
  cls = i.split('/')[-2]
  img = i.split('/')[-1]
  dst = '/content/train_aug/' + cls + '/' + img
  shutil.copy(src, dst)

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
import cv2 as cv
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
Resize_Transformation = transforms.Compose([
   transforms.ToPILImage(), # the transform usually work with PIL images
   transforms.Resize(size=(512, 512)),
])
Horizontal_Flipping_Transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=1) 
])
Vertical_Flipping_Transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomVerticalFlip(p=1) 
])
Color_Transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ColorJitter(brightness=(0.1,0.6), contrast=1,saturation=0, hue=0.4)
])
Contrast_Transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomAutocontrast(p=1)
])
Rotate_Transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(degrees=66)
])

In [ ]:
def prob():
  return random.choice([0, 1, 2])
def augment(img_pth):
  img = Image.open(img_pth)
  # display(img)
  img = np.array(img)
  # save_pth = img_pth.replace('clswise', 'clswise_aug')
  save_pth = img_pth
  save_pth = save_pth[:-4]+'_'+'.png'
  x=0
  if(prob()):
    aug_img = Resize_Transformation(img)
    save_pth = save_pth.split('.')[0]+str(x)+'.png'
    x+=1
    aug_img.save(save_pth)
    # display(aug_img)
  if(prob()):
    aug_img = Horizontal_Flipping_Transformation(img)
    save_pth = save_pth.split('.')[0]+str(x)+'.png'
    x+=1
    aug_img.save(save_pth)
    # display(aug_img)
  if(prob()):
    aug_img = Vertical_Flipping_Transformation(img)
    save_pth = save_pth.split('.')[0]+str(x)+'.png'
    x+=1
    aug_img.save(save_pth)
    # display(aug_img)
  if(prob()):
    aug_img = Color_Transformation(img)
    save_pth = save_pth.split('.')[0]+str(x)+'.png'
    x+=1
    aug_img.save(save_pth)
    # display(aug_img)
  if(prob()):
    aug_img = Rotate_Transformation(img)
    save_pth = save_pth.split('.')[0]+str(x)+'.png'
    x+=1
    aug_img.save(save_pth)
    # display(aug_img)
  if(prob()):
    aug_img = Contrast_Transformation(img)
    save_pth = save_pth.split('.')[0]+str(x)+'.png'
    x+=1
    aug_img.save(save_pth)
    # display(aug_img)

In [ ]:
def get_len_aug(type):
  print(len(glob('/content/train_aug/'+type+'/*')))

def get_len(type):
  print(len(glob('/content/train/'+type+'/*')))

In [ ]:
cls_ = ['com', 'hoa', 'res']

for i in cls_:
  get_len_aug(i)

1255
628
334


In [ ]:
for i in tqdm(glob('/content/train_aug/res/*')):
  augment(i)

for i in tqdm(glob('/content/train_aug/hoa/*')):
  augment(i)

  0%|          | 0/334 [00:00<?, ?it/s]

  0%|          | 0/628 [00:00<?, ?it/s]

In [ ]:
cls_ = ['com', 'hoa', 'res']

for i in cls_:
  get_len_aug(i)

1255
3180
1671


In [ ]:
dl_hoa = glob('/content/train_aug/hoa/*')
random.shuffle(dl_hoa)

dl_res = glob('/content/train_aug/res/*')
random.shuffle(dl_res)

In [ ]:
for i in dl_hoa[0:1400]:
  os.remove(i)

In [ ]:
cls_ = ['com', 'hoa', 'res']

for i in cls_:
  get_len_aug(i)

1255
1780
1671


# Training on EfficientNet-B0

In [ ]:
# !pip install timm
!git clone https://github.com/Padam-Sharma/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 11630, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 11630 (delta 0), reused 0 (delta 0), pack-reused 11627
Receiving objects: 100% (11630/11630), 20.98 MiB | 28.11 MiB/s, done.
Resolving deltas: 100% (8579/8579), done.


In [ ]:
%cd pytorch-image-models

/content/pytorch-image-models


In [ ]:
!pip install -q -e .

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.0 MB/s eta 0:00:00
  Building editable for timm (pyproject.toml) ... done


In [ ]:
!pip install -q focal_loss_torch

In [ ]:
pop_len = len(glob('/content/train/*/*'))
weights = [pop_len/(3*len(glob(i+'/*'))) for i in sorted(glob('/content/train/*'))]

In [ ]:
weights

[0.5888446215139442, 1.1767515923566878, 2.212574850299401]

In [ ]:
!python train.py --data-dir '/content/train_aug' --model 'tf_efficientnet_b0' --pretrained --batch-size 8 --img-size 512 --val-split 0.2 --epochs 30 --num-classes 3

Streaming output truncated to the last 5000 lines.
   macro avg       0.95      0.95      0.95      4008
weighted avg       0.95      0.95      0.95      4008

Train: 12 [ 500/588 ( 85%)]  Loss: 0.1535 (0.0961)  Time: 0.264s,   30.30/s  (0.224s,   35.75/s)  LR: 2.045e-03  
Conf Mat:
 [[ 961   27    6]
 [  70 1461   11]
 [  52   31 1389]]
Data: 0.034 (0.020)
              precision    recall  f1-score   support

           0       0.96      0.89      0.92      1188
           1       0.95      0.96      0.96      1659
           2       0.94      0.99      0.96      1561

    accuracy                           0.95      4408
   macro avg       0.95      0.94      0.95      4408
weighted avg       0.95      0.95      0.95      4408

Train: 12 [ 550/588 ( 94%)]  Loss: 0.02180 (0.101)  Time: 0.193s,   41.41/s  (0.222s,   36.01/s)  LR: 2.045e-03  
Conf Mat:
 [[1055   30   11]
 [  74 1595   12]
 [  59   34 1538]]
Data: 0.006 (0.019)
              precision    recall  f1-score   support

    

In [ ]:
!python '/content/pytorch-image-models/val.py' --data-dir '/content/test' --model 'tf_efficientnet_b0' --checkpoint '/content/pytorch-image-models/output/train/20230504-095358-tf_efficientnet_b0-512/model_best.pth.tar' --img-size 512 --batch-size 1 --num-classes 3 --log-freq 1

Streaming output truncated to the last 5000 lines.
 [ 3  0  0]
 [ 1  0  0]]
classification report 
               precision    recall  f1-score   support

           0       1.00      0.95      0.97        80
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0

    accuracy                           0.95        80
   macro avg       0.33      0.32      0.32        80
weighted avg       1.00      0.95      0.97        80
Acc@1: 100.000 ( 95.000)  Acc@5: 100.000 (100.000)  
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in